In [13]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import struct

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
FNAME = '/project/druhe/SB320-202012281240-lba_outer.vis'

MAGIC = '0x000000003B98F002'
int(MAGIC, 0)

999878658

In [11]:
NANT = 576
SUB = 320
NCHAN = 1
POL2REC = 4
LEN_HDR = 512
NBLINE = NANT * (NANT + 1) // 2
PAYLOAD = NBLINE * NCHAN * POL2REC * 8
RECSIZE = PAYLOAD + LEN_HDR

fsize = os.path.getsize(FNAME)
nrec = fsize // RECSIZE
nrec

418

In [14]:
def read_vis(f):
    s = f.read(RECSIZE)

#     header = s[:512]
    data = s[512:]

    magic, = struct.unpack("<Q", s[0:8])
    magic = np.uint32(magic)
    assert int(magic) == int(MAGIC, 0)

    data = np.reshape(np.frombuffer(data, dtype=np.complex64), (NBLINE, NCHAN, POL2REC))
    return data

In [19]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
# xx, xy, yx, yy
with open(FNAME, 'rb') as f:

    f.seek(0 * RECSIZE)

    vis = read_vis(f)
data = vis.copy()
data.shape

(166176, 1, 4)

In [29]:
from parse_lba_antennafield import parse_lba_antennafield
L = parse_lba_antennafield('../antennafields/a12-AntennaField.conf', [SUB])

from scipy.constants import c
meters_to_wavelengths = 2 * SUB * 2e8 / (1024 * c)   # frequency over speed is wavelength
L_2d = L[:, :2]
distances = meters_to_wavelengths * (L_2d[:, None] - L_2d)

In [32]:
res = 2300
dx = 1.0 / res
out_ax = [(dx, res), (dx, res)]

L = np.linspace(-1, 1, res)
M = np.linspace(-1, 1, res)
mask = np.ones((res, res))
xv, yv = np.meshgrid(L, M)

u, v = distances[np.tril_indices(576)].T

In [31]:
image = data.copy()

im = image.mean(axis=(-1, -2))

In [63]:
def get_beta(W, alpha):

    # see Beatty et al. (2005)
    beta = np.pi*np.sqrt((W*W/alpha/alpha)*(alpha - 0.5)*(alpha - 0.5) - 0.8)

    return beta


In [68]:
def gcf_kaiser(k, Dk, beta):

    temp3 = 2.*k/Dk

    if (1 - temp3)*(1 + temp3) < -1e-12:
#        print "There is an issue with the gridding code!"
        raise Exception("There is an issue with the gridding code!")

    temp3 = np.sqrt(abs((1 - temp3)*(1 + temp3)))

    temp3 = beta*temp3

#    cdef double C = (1./Dk)*gsl_sf_bessel_I0(temp3)/gsl_sf_bessel_I0(beta)
    C = gsl_sf_bessel_I0(temp3)/gsl_sf_bessel_I0(beta)

    return C


In [ ]:
import math

def grid_1d_from_2d(
    x,
    vis,
    dx,
    W,
    beta,
    y,
    x2,
    vis2,
    y2):

        """
        Grid the data in w, Qvix, Uvis in 1D (x) and duplicate orthogonal axes
        """
        N = len(x)

        Dx = W*dx

#         cdef Py_ssize_t indx, xndx, kndx

#         cdef double xval, yval, xref, xg, gcf_val

#         cdef CTYPE_t visval

        for indx in range(N):

            visval = vis[indx]

            xval = x[indx]
            yval = y[indx]

            xref = math.ceil((xval - 0.5*W*dx)/dx)*dx

            for xndx in range(W):

                xg = xref + xndx*dx

                kndx = indx*W + xndx

                gcf_val = gcf_kaiser(xg-xval, Dx, beta)

                vis2[kndx] = visval*gcf_val

                x2[kndx] = xg
                y2[kndx] = yval


In [64]:
def grid_2d(u, v, vis, du, Nu, umin, dv, Nv, vmin, alpha, W, hflag_u, hflag_v):
    nvis = len(u)
    
    tv1 = np.zeros(W)
    tu1 = np.zeros(W)
    tvis1 = np.zeros(W)
    
    beta = get_beta(W, alpha)

    for i in range(nvis):
        
        su = u[i]
        sv = v[i]
        
        svis = vis[i]
        
        grid_1d_from_2d(su, svis, du, W, beta, sv, tu1, tvis1, tv1)
        

In [65]:
def pre_grid(out_ax, alpha=1.5):
    dx = out_ax[0][0]
    Nx = out_ax[0][1]
    xmin = 0
    du = 1 / dx / Nx / alpha
    Nu = int(alpha * Nx)
    umin = 0

    dy = out_ax[1][0]
    Ny = out_ax[1][1]
    ymin = 0
    dv = 1 / dx / Nx / alpha
    Nv = int(alpha * Ny)
    vmin = 0
    
    do_preshift = True
    if do_preshift:
        vmin = -0.5 * Nv * dv
        umin = -0.5 * Nu * du
            
    do_postshift = True
    if do_postshift:
        xmin = -0.5 * Nx * dx
        ymin = -0.5 * Ny * dy
        
    print(xmin, ymin, Ny, dy)
    
    return du, Nu, umin, dv, Nv, vmin
        
    
    

In [66]:
alpha = 1.5
W = 6
du, Nu, umin, dv, Nv, vmin = pre_grid(out_ax, alpha)

-0.5 -0.5 2300 0.0004347826086956522


In [67]:
grid_2d(u, v, im, du, Nu, umin, dv, Nv, vmin, alpha, W, True, True)

12.248183003880952
